In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df=pd.read_csv('../data/processed_train_filled.csv')

In [ ]:
edu_df = df['education'].map({'illiterate':0,'basic.4y':1,'basic.6y':2,'basic.9y':3,
                                  'high.school':4,'professional.course':5,'university.degree':6})
job_df = df['job'].map({'unemployed': 0,'student': 0,'retired': 0,'housemaid': 0,'entrepreneur': 1,
                              'self-employed': 1,'blue-collar': 2,'services': 3,'technician': 4,'admin.': 5,'management': 5})
def_df = df['default'].map({'yes':1,'no':0})
housing_df = df['housing'].map({'yes':1,'no':0})
loan_df = df['loan'].map({'yes':1,'no':0})
debt_df = pd.DataFrame(columns=['debt'])
debt_df['debt'] = def_df + housing_df + loan_df
train_df = pd.concat([edu_df, job_df, debt_df], axis=1)

In [ ]:
mon_df = df['month'].map({'mar':3, 'apr':4, 'may':5, 'jun':6, 'jul':7, 'aug':8, 'sep':9, 'oct':10, 'nov':11, 'dec':12})
day_df = df['day_of_week'].map({'mon':1, 'tue':2, 'wed':3, 'thu':4, 'fri':5})
train_df = pd.concat([train_df, mon_df, day_df], axis=1)

In [ ]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler()
numeric_X = df[['age', 'campaign', 'pdays', 'previous', 'duration']]
train_df = pd.concat([train_df, pd.DataFrame(scaler.fit_transform(numeric_X), columns = numeric_X.columns)], axis=1)
train_df

In [ ]:
y = df['subscribe'].map({'yes':1,'no':0})

In [ ]:
# import train_test split
from sklearn.model_selection import train_test_split
# split data into train and test, 80% in training and 20% in testing
X_train, X_test, y_train, y_test = train_test_split(train_df, y, test_size=0.2,
                                                    random_state = 99)

In [ ]:
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

X_with_intercept = sm.add_constant(X_train)
stats_model = sm.OLS(y_train, X_with_intercept)
results = stats_model.fit()

print("P-values:")
print(results.pvalues)


In [ ]:
coeff_df = pd.DataFrame(lin_reg.coef_, train_df.columns, columns=['Coefficient'])
coeff_df